In [ ]:
import CompDoobTransform as cdt
import time
import math
import torch
import numpy as np
import matplotlib.pyplot as plt
from CompDoobTransform.utils import negative_binomial_logpdf
from torch.distributions.gamma import Gamma
plt.style.use('ggplot')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Computing on ' + str(device))

In [ ]:
# dict for objects relating to latent state process
state = {}

# dimension of state 
d = 1 
state['dim'] = d

# model parameters 
theta = torch.tensor([2.397, 4.429e-03, 0.840, 17.631], device = device)

# drift of diffusion (after Lamperti transformation)
b_constant = theta[0] / theta[2] 
b_factor = theta[1] / theta[2]
b = lambda x: b_constant - b_factor * torch.exp(theta[2] * x) # drift
state['drift'] = b

# diffusion coefficient of diffusion (after Lamperti transformation)
sigma = torch.tensor(1.0, device = device) # diffusion coefficient
state['sigma'] = sigma

# stationary distribution (before Lamperti transformation)
alpha = 2.0 * (0.5 * theta[2]**2 + theta[0]) / theta[2]**2 - 1.0
beta = 2.0 * theta[1] / theta[2]**2
stationary_distribution = Gamma(alpha, beta)

# simulate initial states (from stationary distribution)
initial = lambda N: torch.log(stationary_distribution.sample((N, 1))) / theta[2]
state['initial'] = initial

# time interval
T = torch.tensor(1.0, device = device) 
state['terminal_time'] = T

In [ ]:
# dict for objects relating to observations
obs = {}

# dimension of observation
p = 1
obs['dim'] = p

# log-observation density
obs_log_density = lambda x, y: negative_binomial_logpdf(y, theta[3], torch.exp(theta[2] * x)) 
obs['log_density'] = obs_log_density

# simulate observations
def simulate_obs(x, theta3):
    size = theta3.numpy()
    prob = (theta3 / (theta3 + torch.exp(theta[2] * x))).numpy()
    out = torch.tensor(np.random.negative_binomial(n = size, p = prob))
    return out

observation = lambda N: simulate_obs(initial(N), theta[3])
obs['observation'] = observation

In [ ]:
# learning standardization
N_large = 100000
X = initial(N_large)
Y = torch.tensor(observation(N_large), dtype = torch.float32)

# means and standard deviations
standardization = {'x_mean': torch.mean(X, 0), 
                   'x_std': torch.std(X, 0), 
                   'y_mean': torch.mean(Y, 0), 
                   'y_std': torch.std(Y, 0)}

print(standardization)

# observation mean and variance at stationarity
obs_mean = torch.mean(torch.exp(theta[2] * X))
obs_var = obs_mean + obs_mean**2  / theta[3]

In [ ]:
# time-discretization settings
M = 50 # number of time steps

# V0 and Z neural network configuration
V0_net_config = {'layers': [16], 'standardization': standardization}
Z_net_config = {'layers': [d+16], 'standardization': standardization}
net_config = {'V0': V0_net_config, 'Z': Z_net_config}

# optimization configuration
I = 2000
optim_config = {'minibatch': 100, 
                'num_obs_per_batch': 10, 
                'num_iterations': I,
                'learning_rate' : 0.01, 
                'initial_required' : True}

In [ ]:
# create model instance
model_static = cdt.core.model(state, obs, M, net_config, device = 'cpu')

# static training
time_start = time.time() 
model_static.train_standard(optim_config)
time_end = time.time()
time_elapsed = time_end - time_start
print("Training time (secs): " + str(time_elapsed))

In [ ]:
# create model instance
model = cdt.core.model(state, obs, M, net_config, device = 'cpu')

# iterative training
time_start = time.time() 
model.train_iterative(optim_config)
time_end = time.time()
time_elapsed = time_end - time_start
print("Training time (secs): " + str(time_elapsed))

In [ ]:
# plot loss over optimization iterations
plt.figure()
plt.plot(torch.arange(I), model_static.loss.to('cpu'), '-')
plt.plot(torch.arange(I), model.loss.to('cpu'), '-')
plt.xlabel('iteration', fontsize = 15)
plt.ylabel('loss', fontsize = 15)
plt.legend(['Static CDT', 'Iterative CDT'], fontsize = 15)
plt.show()

In [ ]:
# guided intermediate resampling filter
inverse_temperature = torch.linspace(0.0, 1.0, M+1) # linear schedule

def guiding_initial(x, y, p):
    guiding = inverse_temperature[0]**p * obs_log_density(x,y)
    return guiding

def guiding_intermediate(m, x, x_next, y, p):
    log_potential = inverse_temperature[m-1]**p * obs_log_density(x,y)
    log_potential_next = inverse_temperature[m]**p * obs_log_density(x_next,y) 
    guiding = log_potential_next - log_potential
    return guiding

def guiding_obs_time(m, x, x_next, y, y_next, p):
    guiding = guiding_intermediate(m, x, x_next, y, p) + guiding_initial(x_next, y_next, p)
    return guiding

guiding_linear = {}
guiding_linear['initial'] = lambda x, y: guiding_initial(x, y, 1.0)
guiding_linear['intermediate'] = lambda m, x, x_next, y: guiding_intermediate(m, x, x_next, y, 1.0)
guiding_linear['obs_time'] = lambda m, x, x_next, y, y_next: guiding_obs_time(m, x, x_next, y, y_next, 1.0)

guiding_quadratic = {}
guiding_quadratic['initial'] = lambda x, y: guiding_initial(x, y, 2.0)
guiding_quadratic['intermediate'] = lambda m, x, x_next, y: guiding_intermediate(m, x, x_next, y, 2.0)
guiding_quadratic['obs_time'] = lambda m, x, x_next, y, y_next: guiding_obs_time(m, x, x_next, y, y_next, 2.0)

In [ ]:
# repeat particle filters
multiplier_level = list(range(1,7)) # controls level of misspecification
num_multiplier = len(multiplier_level)
K = 100 # number of observations
R = 100 # number of repeats
N = 2**6 # number of particles
BPF = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}
APF = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}
APFF = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}
GIRF1 = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}
GIRF2 = {'ess' : torch.zeros(num_multiplier, R), 'log_estimate' : torch.zeros(num_multiplier, R)}

for i in range(num_multiplier):
    # level of misspecification
    multiplier = float(multiplier_level[i])
    implied_theta3 = obs_mean**2 / (multiplier**2 * obs_var - obs_mean)

    # simulate latent process and observations
    X0 = initial(1)
    X = torch.zeros(K+1, d)
    X[0,:] = X0.clone()
    Y = torch.zeros(K, p)
    for k in range(K):
        X[k+1,:] = model.simulate_diffusion(X[k,:])
        Y[k,:] = simulate_obs(X[k+1,:], implied_theta3)
    
    for r in range(R):
        # run particle filters
        BPF_output = model.run_BPF(X0.repeat((N,1)), Y, N)
        APF_output = model.run_APF(X0.repeat((N,1)), Y, N)
        APFF_output = model_static.run_APF(X0.repeat((N,1)), Y, N)
        GIRF1_output = model.run_GIRF(X0.repeat((N,1)), Y, N, guiding_linear)
        GIRF2_output = model.run_GIRF(X0.repeat((N,1)), Y, N, guiding_quadratic)

        # save average ESS%
        BPF_ESS = torch.mean(BPF_output['ess'] * 100 / N)
        APF_ESS = torch.mean(APF_output['ess'] * 100 / N)
        APFF_ESS = torch.mean(APFF_output['ess'] * 100 / N)
        GIRF1_ESS = torch.mean(GIRF1_output['ess'] * 100 / N)
        GIRF2_ESS = torch.mean(GIRF2_output['ess'] * 100 / N)
        BPF['ess'][i,r] = BPF_ESS
        APF['ess'][i,r] = APF_ESS
        APFF['ess'][i,r] = APFF_ESS
        GIRF1['ess'][i,r] = GIRF1_ESS
        GIRF2['ess'][i,r] = GIRF2_ESS

        # save log-likelihood estimates
        BPF_log_estimate = BPF_output['log_norm_const'][-1]
        APF_log_estimate = APF_output['log_norm_const'][-1]
        APFF_log_estimate = APFF_output['log_norm_const'][-1]
        GIRF1_log_estimate = GIRF1_output['log_norm_const'][-1]
        GIRF2_log_estimate = GIRF2_output['log_norm_const'][-1]
        BPF['log_estimate'][i,r] = BPF_log_estimate
        APF['log_estimate'][i,r] = APF_log_estimate
        APFF['log_estimate'][i,r] = APFF_log_estimate
        GIRF1['log_estimate'][i,r] = GIRF1_log_estimate
        GIRF2['log_estimate'][i,r] = GIRF2_log_estimate

        # print output
        print('Multipler: ' + str(multiplier) + ' Repeat: ' + str(r)) 
        print('Implied theta3: ' + str(implied_theta3))
        print('BPF ESS%: ' + str(BPF_ESS))
        print('APF ESS%: ' + str(APF_ESS)) 
        print('APFF ESS%: ' + str(APFF_ESS)) 
        print('GIRF1 ESS%: ' + str(GIRF1_ESS)) 
        print('GIRF2 ESS%: ' + str(GIRF2_ESS)) 
        print('BPF log-estimate: ' + str(BPF_log_estimate))
        print('APF log-estimate: ' + str(APF_log_estimate))
        print('APFF log-estimate: ' + str(APFF_log_estimate))
        print('GIRF1 log-estimate: ' + str(GIRF1_log_estimate))
        print('GIRF2 log-estimate: ' + str(GIRF2_log_estimate))

# save results
results = {'BPF' : BPF, 'APF' : APF, 'APFF' : APFF, 'GIRF1' : GIRF1, 'GIRF2' : GIRF2}
torch.save(results, 'logistic_robust.pt')